In [20]:
import sys
import tensorflow as tf
import torch
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

In [21]:
print("torch version",torch.__version__)
print("tensorflow version", tf.__version__)
print("tf gpu", tf.config.list_physical_devices('GPU'))

torch.ones(1)+ torch.ones(1)

torch version 1.9.0
tensorflow version 2.10.0
tf gpu [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


tensor([2.])

In [22]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Num GPUs Available:  1
Tensor Flow Version: 2.10.0
Keras Version: 2.10.0

Python 3.9.12 (main, Apr  5 2022, 01:52:34) 
[Clang 12.0.0 ]
Pandas 1.4.4
Scikit-Learn 1.1.3
GPU is available


In [7]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.python.framework.ops import disable_eager_execution

In [8]:
disable_eager_execution()

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /Users/andy/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [9]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

In [10]:
batch_size = 128

In [11]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [12]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [14]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [17]:
model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

Train on 469 steps, validate on 79 steps
Epoch 1/12
469/469 [==============================] - 11s 21ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0036 - accuracy: 0.9988 - val_loss: 0.0501 - val_accuracy: 0.9900
Epoch 2/12
469/469 [==============================] - 10s 20ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0011 - accuracy: 0.9995 - val_loss: 0.0582 - val_accuracy: 0.9884
Epoch 3/12
469/469 [==============================] - 11s 20ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0019 - accuracy: 0.9993 - val_loss: 0.0535 - val_accuracy: 0.9916
Epoch 4/12
469/469 [==============================] - 10s 20ms/step - batch: 234.0000 - size: 1.0000 - loss: 9.2941e-04 - accuracy: 0.9996 - val_loss: 0.0496 - val_accuracy: 0.9916
Epoch 5/12
469/469 [==============================] - 11s 20ms/step - batch: 234.0000 - size: 1.0000 - loss: 8.3051e-04 - accuracy: 0.9997 - val_loss: 0.0534 - val_accuracy: 0.9907
Epoch 6/12
469/469 [==============================] - 10s 20ms/ste